<a href="https://colab.research.google.com/github/piesauce/oreilly-llm-course/blob/main/context_engineering_oreilly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Context Engineering for Long Conversations (Dynamic Context Swapping)

This Colab notebook demonstrates a practical **context engineering** loop:

- Keep a long-running conversation with an OpenAI model
- Maintain multiple context layers (**instructions**, **running summary**, **memory**, **retrieved docs**, **recent turns**)
- Automatically **swap data in/out of the model context** so the chat can continue for a long time without dumping the entire transcript each turn
- Inspect **the exact context payload** being sent to the model at any time (and even diff it turn-to-turn)

> Safety note: don’t paste secrets or sensitive personal data into prompts unless you’re comfortable sending them to the API.


## 1) Install & set up

This notebook uses the OpenAI **Responses API** via the official Python SDK.

- You'll set `OPENAI_API_KEY` (you can paste it, or use Colab Secrets / environment variables).
- Then we'll build a `ContextEngine` that:
  - stores conversation turns
  - summarizes older turns into a running summary
  - stores durable "memory" facts
  - ingests large documents and retrieves only relevant snippets
  - assembles an *explicit* context packet each turn and shows it to you


In [ ]:
# Colab: install the SDK + a few tiny helpers
!pip -q install --upgrade openai pydantic scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.6/463.6 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 40.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.50.0 requires pydantic<=2.12.3,>=2.0, but you have pydantic 2.12.5 which is incompatible.


In [ ]:
import os, getpass
from openai import OpenAI

# Option A: already set as an env var
if not os.environ.get("OPENAI_API_KEY"):
    # Option B: paste it here (input is hidden)
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter OPENAI_API_KEY: ")

client = OpenAI()
print("OpenAI client ready.")


## 2) Configuration

Choose model IDs you have access to. The defaults below use a recent general-purpose model.

We also set a **max input token budget** for what we send each turn (instructions + context packet + recent messages).
The engine will compact/summarize and/or reduce retrieved snippets to stay under the budget.


In [ ]:
# --- Models ---
MODEL = "gpt-5.2"           # main chat model
SUMMARIZER_MODEL = MODEL    # used to summarize older chat history
MEMORY_MODEL = MODEL        # used to extract durable memory facts

# --- Context policy knobs ---
MAX_INPUT_TOKENS = 6000         # how much context we *send* each turn (not the model's full context window)
KEEP_LAST_MESSAGES = 10         # keep this many of the most recent messages verbatim (user+assistant count)
RETRIEVED_SNIPPETS = 4          # how many doc/memory snippets to retrieve and swap into context each turn
PINNED_MEMORY_LIMIT = 20        # cap pinned memories (oldest/least important are dropped)
AUTO_MEMORY = True              # let the model extract durable "memory" facts from turns
TOKEN_COUNTING = True           # uses the token counting endpoint (extra API call) for exact input sizes
SHOW_CONTEXT_EACH_TURN = False  # if True, prints context after every user turn

print("Config loaded.")


## 3) The ContextEngine

This is the heart of the demo.

### Layers we manage

1. **Base instructions** (stable)
2. **Running summary** of older conversation turns (compressed)
3. **Durable memory** (facts/preferences/goals)
4. **Large external docs** (ingested once, then retrieved by relevance)
5. **Recent chat turns** (kept verbatim)

Each turn we assemble an explicit *context packet* and pass it via `instructions=...` along with the recent messages via `input=[...]`.
We also store the last payload so you can call `engine.show_context()` any time.


In [ ]:
from __future__ import annotations

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Tuple
import difflib
import json
import re
import textwrap
import time
import uuid

from pydantic import BaseModel, Field
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# -----------------------------
# Small utilities
# -----------------------------

def _now_iso() -> str:
    return time.strftime("%Y-%m-%dT%H:%M:%SZ", time.gmtime())

def chunk_text(text: str, max_chars: int = 1200, overlap: int = 150) -> List[str]:
    """Chunk large text into overlapping slices (simple + robust for demos)."""
    text = text.strip()
    if not text:
        return []
    if len(text) <= max_chars:
        return [text]

    chunks = []
    start = 0
    while start < len(text):
        end = min(len(text), start + max_chars)
        chunk = text[start:end].strip()
        if chunk:
            chunks.append(chunk)
        if end >= len(text):
            break
        start = max(0, end - overlap)
    return chunks

def format_messages(messages: List[Dict[str, str]]) -> str:
    lines = []
    for m in messages:
        role = m.get("role", "?")
        content = m.get("content", "")
        lines.append(f"{role.upper()}: {content}")
    return "\n".join(lines)

def safe_truncate(s: str, max_chars: int = 2500) -> str:
    s = s or ""
    if len(s) <= max_chars:
        return s
    head = s[: max_chars // 2]
    tail = s[-max_chars // 2 :]
    return head + "\n... (truncated) ...\n" + tail


# -----------------------------
# Structured memory extraction schema
# -----------------------------

class MemoryFact(BaseModel):
    """A single durable fact worth remembering."""
    category: str = Field(
        description="One of: identity, preference, project, constraint, other"
    )
    text: str = Field(description="The memory as a short, standalone sentence.")
    importance: int = Field(ge=1, le=5, description="1=low, 5=high")

class MemoryDelta(BaseModel):
    """What to add to memory after a turn."""
    add: List[MemoryFact] = Field(default_factory=list)


# -----------------------------
# Core engine
# -----------------------------

@dataclass
class KnowledgeChunk:
    id: str
    kind: str   # "doc" or "memory"
    text: str
    source: str
    created_at: str = field(default_factory=_now_iso)
    metadata: Dict[str, Any] = field(default_factory=dict)


class ContextEngine:
    """A tiny, inspectable context-management layer for long conversations.

    Key idea: keep the conversation going by *swapping* context in/out:
      - Summarize older turns into a running summary (swap out transcript -> swap in summary)
      - Store durable memory facts (swap in small facts instead of repeating)
      - Ingest large docs and retrieve only the top-K relevant snippets per turn
    """

    def __init__(
        self,
        client,
        model: str,
        summarizer_model: Optional[str] = None,
        memory_model: Optional[str] = None,
        max_input_tokens: int = 6000,
        keep_last_messages: int = 10,
        retrieved_snippets: int = 4,
        pinned_memory_limit: int = 20,
        auto_memory: bool = True,
        token_counting: bool = True,
        show_context_each_turn: bool = False,
    ):
        self.client = client
        self.model = model
        self.summarizer_model = summarizer_model or model
        self.memory_model = memory_model or model

        # policy knobs
        self.max_input_tokens = max_input_tokens
        self.keep_last_messages = keep_last_messages
        self.retrieved_snippets = retrieved_snippets
        self.pinned_memory_limit = pinned_memory_limit
        self.auto_memory = auto_memory
        self.token_counting = token_counting
        self.show_context_each_turn = show_context_each_turn

        # stable instruction (you can edit this!)
        self.base_instructions = (
            "You are a helpful assistant.\n"
            "Be accurate. Ask clarifying questions when needed.\n"
            "When given a Context Packet, use it as your source of truth for user preferences and prior decisions."
        )

        # mutable state
        self.summary: str = ""
        self.recent_messages: List[Dict[str, str]] = []   # [{role, content}]
        self.memories: List[Dict[str, Any]] = []          # durable facts (some pinned)
        self.docs: List[KnowledgeChunk] = []              # large doc chunks
        self._vectorizer: Optional[TfidfVectorizer] = None
        self._matrix = None
        self._knowledge: List[KnowledgeChunk] = []        # docs + memories (as chunks)
        self._last_payload: Optional[Dict[str, Any]] = None
        self._prev_payload: Optional[Dict[str, Any]] = None

        self._rebuild_knowledge_index()

    # ---------- public: ingest / remember ----------

    def ingest_document(self, text: str, source: str = "user_doc", max_chars: int = 1200) -> List[str]:
        """Ingest a large document once; we will retrieve only relevant chunks later."""
        chunks = chunk_text(text, max_chars=max_chars)
        new_ids = []
        for i, ch in enumerate(chunks):
            cid = f"doc_{uuid.uuid4().hex[:10]}"
            self.docs.append(
                KnowledgeChunk(
                    id=cid,
                    kind="doc",
                    text=ch,
                    source=source,
                    metadata={"chunk_index": i, "chunks_total": len(chunks)},
                )
            )
            new_ids.append(cid)
        self._rebuild_knowledge_index()
        return new_ids

    def remember(self, text: str, category: str = "other", importance: int = 3, pinned: Optional[bool] = None):
        """Add a durable memory fact.

        If pinned is None, we auto-pin high-importance identity/preferences.
        """
        text = (text or "").strip()
        if not text:
            return

        if pinned is None:
            pinned = (category in {"identity", "preference", "constraint"} and importance >= 4)

        mid = f"mem_{uuid.uuid4().hex[:10]}"
        item = {
            "id": mid,
            "kind": "memory",
            "category": category,
            "text": text,
            "importance": int(importance),
            "pinned": bool(pinned),
            "created_at": _now_iso(),
        }
        self.memories.append(item)

        # enforce pinned cap (drop oldest low-importance pinned first)
        self._enforce_pinned_cap()

        self._rebuild_knowledge_index()

    def forget(self, pattern: str) -> int:
        """Remove memories whose text matches a regex/pattern (case-insensitive)."""
        pattern = (pattern or "").strip()
        if not pattern:
            return 0
        rx = re.compile(pattern, re.IGNORECASE)
        before = len(self.memories)
        self.memories = [m for m in self.memories if not rx.search(m.get("text", ""))]
        removed = before - len(self.memories)
        if removed:
            self._rebuild_knowledge_index()
        return removed

    def reset_conversation(self, keep_memory: bool = True, keep_docs: bool = True):
        self.summary = ""
        self.recent_messages = []
        self._prev_payload = None
        self._last_payload = None
        if not keep_memory:
            self.memories = []
        if not keep_docs:
            self.docs = []
        self._rebuild_knowledge_index()

    # ---------- retrieval + context assembly ----------

    def _enforce_pinned_cap(self):
        pinned = [m for m in self.memories if m.get("pinned")]
        if len(pinned) <= self.pinned_memory_limit:
            return
        # sort pinned by (importance asc, created_at asc) and drop until within cap
        pinned_sorted = sorted(pinned, key=lambda m: (m.get("importance", 3), m.get("created_at", "")))
        to_drop = set(m["id"] for m in pinned_sorted[: len(pinned) - self.pinned_memory_limit])
        self.memories = [m for m in self.memories if m["id"] not in to_drop]

    def _rebuild_knowledge_index(self):
        # represent memories as KnowledgeChunks so docs+memories can share the same TF-IDF index
        mem_chunks: List[KnowledgeChunk] = []
        for m in self.memories:
            mem_chunks.append(
                KnowledgeChunk(
                    id=m["id"],
                    kind="memory",
                    text=f"[{m['category'].upper()}] {m['text']}",
                    source="memory",
                    metadata={"importance": m.get("importance", 3), "pinned": m.get("pinned", False)},
                )
            )
        self._knowledge = list(self.docs) + mem_chunks

        if not self._knowledge:
            self._vectorizer = None
            self._matrix = None
            return

        texts = [k.text for k in self._knowledge]
        # small + demo-friendly: rebuild from scratch
        self._vectorizer = TfidfVectorizer(stop_words="english")
        self._matrix = self._vectorizer.fit_transform(texts)

    def retrieve(self, query: str, k: Optional[int] = None) -> List[Tuple[KnowledgeChunk, float]]:
        """Return top-k knowledge chunks relevant to `query`."""
        k = int(k or self.retrieved_snippets)
        query = (query or "").strip()
        if not query or not self._knowledge or self._vectorizer is None or self._matrix is None:
            return []

        qv = self._vectorizer.transform([query])
        sims = cosine_similarity(qv, self._matrix).flatten()
        # get best indices
        best = sims.argsort()[::-1]
        results: List[Tuple[KnowledgeChunk, float]] = []
        for idx in best[: max(k * 3, k)]:  # scan a bit deeper for non-zero sims
            score = float(sims[idx])
            if score <= 0:
                continue
            results.append((self._knowledge[idx], score))
            if len(results) >= k:
                break
        return results

    def _build_context_packet(self, latest_user_message: str) -> Dict[str, Any]:
        """Create a structured context packet we can both show to the user and embed in instructions."""
        retrieved = self.retrieve(latest_user_message, k=self.retrieved_snippets)

        pinned = [m for m in self.memories if m.get("pinned")]
        pinned_sorted = sorted(pinned, key=lambda m: (-m.get("importance", 3), m.get("created_at", "")))

        # Memories that were retrieved by relevance (non-pinned too)
        retrieved_mem = []
        for chunk, score in retrieved:
            if chunk.kind == "memory":
                # look up original memory record
                m = next((mm for mm in self.memories if mm["id"] == chunk.id), None)
                if m:
                    retrieved_mem.append({**m, "score": score})

        # Deduplicate: pinned wins
        pinned_ids = {m["id"] for m in pinned_sorted}
        retrieved_mem = [m for m in retrieved_mem if m["id"] not in pinned_ids]

        retrieved_docs = []
        for chunk, score in retrieved:
            if chunk.kind == "doc":
                retrieved_docs.append({
                    "id": chunk.id,
                    "source": chunk.source,
                    "score": score,
                    "text": chunk.text,
                    "metadata": chunk.metadata,
                })

        packet = {
            "summary": self.summary.strip(),
            "pinned_memory": pinned_sorted,
            "retrieved_memory": retrieved_mem,
            "retrieved_docs": retrieved_docs,
        }
        return packet

    def _render_context_packet_as_text(self, packet: Dict[str, Any]) -> str:
        """Convert packet -> a compact text block that goes into `instructions`."""
        parts = []
        parts.append("## Context Packet (auto-generated each turn)")

        # summary
        summary = packet.get("summary") or ""
        if summary.strip():
            parts.append("### Running summary (older turns)")
            parts.append(summary.strip())
        else:
            parts.append("### Running summary (older turns)")
            parts.append("(none yet)")

        # memory
        def _mem_line(m: Dict[str, Any]) -> str:
            pin = "📌" if m.get("pinned") else ""
            cat = m.get("category", "other")
            imp = m.get("importance", 3)
            return f"- {pin}[{cat}, imp={imp}] {m.get('text','').strip()} (id={m.get('id')})"

        pinned = packet.get("pinned_memory") or []
        retrieved_mem = packet.get("retrieved_memory") or []

        parts.append("### Durable memory (preferences/goals/etc.)")
        if not pinned and not retrieved_mem:
            parts.append("(none)")

        if pinned:
            parts.append("Pinned:")
            parts.extend(_mem_line(m) for m in pinned)

        if retrieved_mem:
            parts.append("Relevant (retrieved):")
            parts.extend(_mem_line(m) for m in retrieved_mem)

        # retrieved docs
        docs = packet.get("retrieved_docs") or []
        parts.append("### Retrieved references (swap-in snippets)")
        if not docs:
            parts.append("(none retrieved for this turn)")
        else:
            for d in docs:
                snippet = safe_truncate(d["text"], max_chars=1200)
                parts.append(f"[doc_id={d['id']} | source={d['source']} | score={d['score']:.3f}]\n" + snippet)

        # rules
        parts.append("### Rules for using the Context Packet")
        parts.append("- Treat Durable memory + Running summary as authoritative for prior user preferences and decisions.")
        parts.append("- If the user request conflicts with memory, ask a clarifying question.")
        parts.append("- Retrieved references may be partial; cite them explicitly if you rely on them.")
        return "\n\n".join(parts).strip()

    def _assemble_instructions_and_input(self, latest_user_message: str) -> Tuple[str, List[Dict[str, str]], Dict[str, Any]]:
        packet = self._build_context_packet(latest_user_message)
        packet_text = self._render_context_packet_as_text(packet)
        instructions = self.base_instructions + "\n\n" + packet_text
        input_items = list(self.recent_messages)  # shallow copy
        debug = {
            "packet": packet,
            "instructions": instructions,
            "input": input_items,
        }
        return instructions, input_items, debug

    # ---------- token counting + compaction ----------

    def _count_input_tokens(self, instructions: str, input_items: List[Dict[str, str]]) -> Optional[int]:
        if not self.token_counting:
            return None
        # NOTE: this is an extra API call
        try:
            r = self.client.responses.input_tokens.count(
                model=self.model,
                instructions=instructions,
                input=input_items,
            )
            return int(r.input_tokens)
        except Exception as e:
            print("[token_counting failed]", repr(e))
            return None

    def _summarize_into_running_summary(self, messages_to_summarize: List[Dict[str, str]]) -> str:
        transcript = format_messages(messages_to_summarize)

        prompt = f"""Update the running summary for a long conversation.

Existing running summary (may be empty):
{self.summary.strip() or "(none)"}

New transcript chunk to incorporate:
{transcript}

Write an updated running summary that:
- keeps user goals, preferences, constraints
- keeps decisions, plans, and important factual context
- omits chit-chat and redundant detail
- is written in plain text (bullets OK)

Return ONLY the updated running summary.
"""

        try:
            resp = self.client.responses.create(
                model=self.summarizer_model,
                instructions="You summarize conversations into compact running summaries.",
                input=prompt,
            )
            return resp.output_text.strip()
        except Exception as e:
            print("[summarization failed]", repr(e))
            # fallback: naive compression
            return safe_truncate((self.summary + "\n" + transcript).strip(), max_chars=2000)

    def _ensure_within_budget(self, latest_user_message: str) -> Tuple[str, List[Dict[str, str]], Dict[str, Any], Optional[int]]:
        """Try to keep the payload <= max_input_tokens by swapping context in/out."""
        # We'll keep trying in this order:
        #   1) summarize older turns (swap transcript -> summary)
        #   2) reduce retrieved snippet count (swap fewer docs in)
        #   3) truncate the running summary if it grows too large

        attempts = 0
        local_retrieved_k = int(self.retrieved_snippets)

        while True:
            instructions, input_items, debug = self._assemble_instructions_and_input(latest_user_message)
            tokens = self._count_input_tokens(instructions, input_items)

            # If token counting is off or fails, we can't enforce strictly.
            if tokens is None:
                debug["note"] = "Token counting disabled/failed; not enforcing budget."
                return instructions, input_items, debug, tokens

            if tokens <= self.max_input_tokens:
                debug["input_tokens"] = tokens
                debug["retrieved_k"] = local_retrieved_k
                return instructions, input_items, debug, tokens

            attempts += 1
            if attempts > 10:
                debug["note"] = "Gave up enforcing budget after many attempts."
                debug["input_tokens"] = tokens
                return instructions, input_items, debug, tokens

            # (1) Summarize older messages if we have more than keep_last_messages
            if len(self.recent_messages) > self.keep_last_messages:
                to_summarize = self.recent_messages[: -self.keep_last_messages]
                self.summary = self._summarize_into_running_summary(to_summarize)
                self.recent_messages = self.recent_messages[-self.keep_last_messages :]
                continue

            # (2) Reduce retrieved snippets
            if local_retrieved_k > 0:
                local_retrieved_k -= 1
                self.retrieved_snippets = local_retrieved_k
                continue

            # (3) Truncate summary as last resort
            if self.summary and len(self.summary) > 1500:
                self.summary = self.summary[-1500:]
                continue

            # (4) As a final fallback, drop oldest message(s)
            if len(self.recent_messages) > 2:
                self.recent_messages = self.recent_messages[2:]
                continue

            # nothing else to do
            debug["note"] = "Unable to reduce further."
            debug["input_tokens"] = tokens
            return instructions, input_items, debug, tokens

    # ---------- auto memory extraction ----------

    def _auto_memory_extract(self, user_text: str, assistant_text: str) -> MemoryDelta:
        """Use the model to extract durable memory facts (structured output)."""
        existing = [
            {"id": m["id"], "category": m["category"], "text": m["text"], "importance": m["importance"], "pinned": m["pinned"]}
            for m in self.memories
        ]

        prompt = f"""You are a memory extraction module for a chat assistant.

Your job: extract ONLY durable, reusable facts worth remembering for future turns.
Examples: identity, long-term goals, stable preferences, constraints, project specs.
Do NOT include ephemeral details (e.g., jokes, one-off numbers, temporary states) unless the user says it is important.

Existing memories (avoid duplicates):
{json.dumps(existing, ensure_ascii=False)}

Latest turn:
USER: {user_text}
ASSISTANT: {assistant_text}

Return a JSON object matching the provided schema.
"""

        # Structured Outputs helper (parse into Pydantic model)
        try:
            resp = self.client.responses.parse(
                model=self.memory_model,
                input=[
                    {"role": "system", "content": "Extract durable memory facts in structured form."},
                    {"role": "user", "content": prompt},
                ],
                text_format=MemoryDelta,
            )
            return resp.output_parsed
        except Exception as e:
            print("[memory extract failed]", repr(e))
            return MemoryDelta(add=[])

    def _apply_memory_delta(self, delta: MemoryDelta):
        for fact in delta.add:
            # normalize
            cat = (fact.category or "other").strip().lower()
            if cat not in {"identity", "preference", "project", "constraint", "other"}:
                cat = "other"
            txt = (fact.text or "").strip()
            if not txt:
                continue

            # avoid near-duplicates
            txt_l = txt.lower()
            if any(txt_l == (m.get("text", "").strip().lower()) for m in self.memories):
                continue

            self.remember(txt, category=cat, importance=int(fact.importance), pinned=None)

    # ---------- main step + inspection ----------

    def step(self, user_text: str) -> str:
        """One conversational turn."""
        user_text = (user_text or "").strip()
        if not user_text:
            return ""

        self._prev_payload = self._last_payload

        # Add user message to the rolling window
        self.recent_messages.append({"role": "user", "content": user_text})

        # Ensure we stay under budget before generating
        instructions, input_items, debug, tokens = self._ensure_within_budget(user_text)

        # Call the model
        resp = self.client.responses.create(
            model=self.model,
            instructions=instructions,
            input=input_items,
        )
        assistant_text = resp.output_text.strip()

        # Add assistant message
        self.recent_messages.append({"role": "assistant", "content": assistant_text})

        # Auto-memory update
        if self.auto_memory:
            delta = self._auto_memory_extract(user_text, assistant_text)
            self._apply_memory_delta(delta)

        # Save payload so we can inspect it anytime
        self._last_payload = {
            "model": self.model,
            "instructions": instructions,
            "input": input_items,
            "input_tokens": tokens,
            "debug": debug,
        }

        if self.show_context_each_turn:
            self.show_context(show_full=False)

        return assistant_text

    def preview_next_payload(self, next_user_message: str) -> Dict[str, Any]:
        """Show what would be sent if `next_user_message` were the next user turn."""
        instructions, input_items, debug = self._assemble_instructions_and_input(next_user_message)
        tokens = self._count_input_tokens(instructions, input_items)
        return {
            "model": self.model,
            "instructions": instructions,
            "input": input_items,
            "input_tokens": tokens,
            "debug": debug,
        }

    def show_context(self, show_full: bool = True, max_chars: int = 4000):
        """Print the *exact* payload we last sent to the model."""
        if not self._last_payload:
            print("(no model call yet)")
            return

        p = self._last_payload
        print("\n" + "="*80)
        print(f"MODEL: {p.get('model')} | input_tokens={p.get('input_tokens')}")
        print("-"*80)
        print("INSTRUCTIONS (what we prepend every turn):")
        print("-"*80)
        instr = p.get("instructions", "")
        print(safe_truncate(instr, max_chars=max_chars if show_full else 1200))
        print("-"*80)
        print("INPUT MESSAGES (recent chat window):")
        for m in p.get("input", []):
            role = m.get("role")
            content = m.get("content", "")
            print(f"[{role}] {safe_truncate(content, max_chars=700 if not show_full else 2000)}")
        print("="*80 + "\n")

    def diff_context(self):
        """Show a diff of instructions between the last two turns."""
        if not self._prev_payload or not self._last_payload:
            print("(need at least two turns)")
            return
        a = (self._prev_payload.get("instructions") or "").splitlines(keepends=True)
        b = (self._last_payload.get("instructions") or "").splitlines(keepends=True)
        diff = difflib.unified_diff(a, b, fromfile="prev_instructions", tofile="curr_instructions")
        print("".join(diff))

    # ---------- interactive loop (optional) ----------

    def chat(self):
        """Simple terminal-like chat loop (works in Colab)."""
        help_text = """Commands:
  /help                 show this help
  /context              print the last payload sent to the model
  /diff                 diff the last two instruction payloads
  /memory               list memories
  /forget <pattern>     delete memories matching regex/pattern
  /summary              print the running summary
  /ingest               ingest a document (paste, end with a single line: END)
  /reset                reset conversation (keeps memory+docs)
  /reset_all            reset conversation + memory + docs
  /exit                 leave chat
"""

        print(help_text)
        while True:
            user = input("you> ").strip()
            if not user:
                continue
            if user in {"/exit", "/quit"}:
                break
            if user == "/help":
                print(help_text)
                continue
            if user == "/context":
                self.show_context(show_full=True)
                continue
            if user == "/diff":
                self.diff_context()
                continue
            if user == "/memory":
                if not self.memories:
                    print("(no memories)")
                else:
                    for m in sorted(self.memories, key=lambda x: (-x.get("importance", 3), x.get("created_at", ""))):
                        pin = "📌" if m.get("pinned") else ""
                        print(f"{pin}{m['id']} [{m['category']}, imp={m['importance']}] {m['text']}")
                continue
            if user.startswith("/forget "):
                pat = user[len("/forget "):].strip()
                n = self.forget(pat)
                print(f"Removed {n} memories.")
                continue
            if user == "/summary":
                print(self.summary.strip() or "(none)")
                continue
            if user == "/reset":
                self.reset_conversation(keep_memory=True, keep_docs=True)
                print("Conversation reset (memory+docs kept).")
                continue
            if user == "/reset_all":
                self.reset_conversation(keep_memory=False, keep_docs=False)
                print("Conversation + memory + docs reset.")
                continue
            if user == "/ingest":
                print("Paste text to ingest. End with a single line: END")
                lines = []
                while True:
                    line = input()
                    if line.strip() == "END":
                        break
                    lines.append(line)
                text = "\n".join(lines).strip()
                if text:
                    ids = self.ingest_document(text, source="pasted_doc")
                    print(f"Ingested {len(ids)} chunks.")
                else:
                    print("(no text ingested)")
                continue

            # normal chat turn
            assistant = self.step(user)
            print("\nassistant>" , assistant, "\n")


## 4) Start chatting

Create an engine, then run `engine.chat()`.

Tip: try *long* conversations, and occasionally call:

- `/context` to see the exact context being sent
- `/diff` to see how the context packet changed between turns (what got swapped in/out)
- `/ingest` to paste a large document and then ask questions about it


In [ ]:
engine = ContextEngine(
    client=client,
    model=MODEL,
    summarizer_model=SUMMARIZER_MODEL,
    memory_model=MEMORY_MODEL,
    max_input_tokens=MAX_INPUT_TOKENS,
    keep_last_messages=KEEP_LAST_MESSAGES,
    retrieved_snippets=RETRIEVED_SNIPPETS,
    pinned_memory_limit=PINNED_MEMORY_LIMIT,
    auto_memory=AUTO_MEMORY,
    token_counting=TOKEN_COUNTING,
    show_context_each_turn=SHOW_CONTEXT_EACH_TURN,
)

print("Engine ready. Run: engine.chat()")


In [ ]:
engine.chat()

## 5) Optional: a tiny scripted demo

Run this cell to see:

- Document ingestion (large text stored out-of-context)
- Retrieval swapping (only relevant snippets are inserted into the context packet)
- Memory extraction (the assistant stores durable facts)
- Context inspection (we print the last payload)


In [ ]:
# Ingest a "large" doc (for demo purposes)
doc_text = """Context engineering is the practice of carefully selecting what information goes into
a model's context window at each turn. Because context windows are limited, long
conversations require techniques like summarization, retrieval, and memory.

Summarization swaps out older verbatim turns for a compact summary.
Retrieval swaps in only the most relevant external snippets for the current query.
Memory stores durable facts (preferences, goals) to reuse across the session.
"""

engine.ingest_document(doc_text, source="demo_doc")

# A couple turns
print(engine.step("Hi! My name is Sam. Please remember I prefer concise bullet points."))
print(engine.step("Can you explain context engineering?"))

# Inspect what we actually sent
engine.show_context(show_full=False)
